In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
# https://www.imdb.com/chart/top?ref_=nv_mv_250

In [5]:
response = requests.get('https://www.imdb.com/chart/top?ref_=nv_mv_250')
doc = BeautifulSoup(response.text)

movies = doc.find(class_ = 'lister-list').find_all('tr')

In [75]:
rows = []
for movie in movies:
    row = {}
    row['title'] = movie.find(class_ = 'titleColumn').a.text.strip()
    row['rank'] = re.findall(r'(\d+)\..*', movie.find(class_ = 'titleColumn').text.strip())[0]
    row['director'] = re.findall(r'^(.+) \(dir.\).+', movie.find(class_ = 'titleColumn').a['title'])[0]
    row['main_cast'] = re.findall(r'^.+ \(dir.\), (.+)', movie.find(class_ = 'titleColumn').a['title'])[0]
    row['url'] = movie.a['href']
    row['img'] = movie.a.img['src']
    row['year'] = movie.find(class_ = 'secondaryInfo').text.strip('()')
    row['rating'] = movie.find(class_ = 'ratingColumn imdbRating').text.strip()
    rows.append(row)
    
rows

[{'title': 'The Shawshank Redemption',
  'rank': '1',
  'director': 'Frank Darabont',
  'main_cast': 'Tim Robbins, Morgan Freeman',
  'url': '/title/tt0111161/',
  'img': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY67_CR0,0,45,67_AL_.jpg',
  'year': '1994',
  'rating': '9.2'},
 {'title': 'The Godfather',
  'rank': '2',
  'director': 'Francis Ford Coppola',
  'main_cast': 'Marlon Brando, Al Pacino',
  'url': '/title/tt0068646/',
  'img': 'https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY67_CR1,0,45,67_AL_.jpg',
  'year': '1972',
  'rating': '9.2'},
 {'title': 'The Godfather: Part II',
  'rank': '3',
  'director': 'Francis Ford Coppola',
  'main_cast': 'Al Pacino, Robert De Niro',
  'url': '/title/tt0071562/',
  'img': 'https://m.media-amazon.com/images/M/MV5BMWMwMGQzZTItY2JlNC00OWZiLWIyMDctNDk2ZDQ2YjRjMWQ0XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._

In [79]:
df = pd.DataFrame(rows)
df = df[['title', 'rank', 'rating', 'director', 'year', 'main_cast', 'url', 'img']]

In [80]:
df.head()

,title,rank,rating,director,year,main_cast,url,img
0,The Shawshank Redemption,1,9.2,Frank Darabont,1994,"Tim Robbins, Morgan Freeman",/title/tt0111161/,https://m.media-amazon.com/images/M/MV5BMDFkYT...
1,The Godfather,2,9.2,Francis Ford Coppola,1972,"Marlon Brando, Al Pacino",/title/tt0068646/,https://m.media-amazon.com/images/M/MV5BM2MyNj...
2,The Godfather: Part II,3,9.0,Francis Ford Coppola,1974,"Al Pacino, Robert De Niro",/title/tt0071562/,https://m.media-amazon.com/images/M/MV5BMWMwMG...
3,The Dark Knight,4,9.0,Christopher Nolan,2008,"Christian Bale, Heath Ledger",/title/tt0468569/,https://m.media-amazon.com/images/M/MV5BMTMxNT...
4,12 Angry Men,5,8.9,Sidney Lumet,1957,"Henry Fonda, Lee J. Cobb",/title/tt0050083/,https://m.media-amazon.com/images/M/MV5BMWU4N2...


In [83]:
df.to_csv('imdb_top250.csv', index = False)

In [213]:
df = pd.read_csv('imdb_top250.csv')
df.head()

,title,rank,rating,director,year,main_cast,url,img
0,The Shawshank Redemption,1,9.2,Frank Darabont,1994,"Tim Robbins, Morgan Freeman",/title/tt0111161/,https://m.media-amazon.com/images/M/MV5BMDFkYT...
1,The Godfather,2,9.2,Francis Ford Coppola,1972,"Marlon Brando, Al Pacino",/title/tt0068646/,https://m.media-amazon.com/images/M/MV5BM2MyNj...
2,The Godfather: Part II,3,9.0,Francis Ford Coppola,1974,"Al Pacino, Robert De Niro",/title/tt0071562/,https://m.media-amazon.com/images/M/MV5BMWMwMG...
3,The Dark Knight,4,9.0,Christopher Nolan,2008,"Christian Bale, Heath Ledger",/title/tt0468569/,https://m.media-amazon.com/images/M/MV5BMTMxNT...
4,12 Angry Men,5,8.9,Sidney Lumet,1957,"Henry Fonda, Lee J. Cobb",/title/tt0050083/,https://m.media-amazon.com/images/M/MV5BMWU4N2...


In [416]:
def get_info(data):
    url = 'https://www.imdb.com'+ data['url']
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    print('scraping', url)
    row = {}
    bar = doc.find(class_ = 'title_bar_wrapper')
    row['title'] = bar.h1.contents[0].strip()
    row['rating_count'] = bar.find('span',class_ = 'small').text.strip()
    row['runtime'] = bar.time.text.strip()
    row['genre'] = [x.text.strip() for x in bar.findAll('a')[2:-1]]
    row['release_date'] = bar.findAll('a')[-1].text.strip()
    row['rating'] = bar.find(class_ = 'subtext').contents[0].strip()
    summary = doc.find(class_ = 'plot_summary_wrapper')
    row['plot'] = summary.find(class_ = 'summary_text').text.strip()
    row['writers'] = [x.text for x in summary.find(class_ = 'credit_summary_item').find_next_sibling().findAll('a')]
    row['stars'] = [x.text for x in summary.find(class_ = 'credit_summary_item').find_next_sibling().find_next_sibling().findAll('a', limit = 3)]
    row['plot_keywords'] = [x.text.strip() for x in doc.find(class_ = 'see-more inline canwrap').findAll('a', limit = 5)]
    details = doc.find(class_ = 'article', id = 'titleDetails')
    row['country'] = [x.text.strip() for x in details.find('h4', string = 'Country:').find_next_siblings('a')]
    row['lang'] = [x.text.strip() for x in details.find('h4', string = 'Language:').find_next_siblings('a')]
    try:
        row['film_location'] = details.find('h4', string = 'Filming Locations:').find_next_sibling('a').text.strip()
    except:
        pass
    try: 
        row['budget'] = details.find('h4', string = 'Budget:').next_element.next_element.strip()
    except:
        pass
    try:
        row['us_gross'] = details.find('h4', string = 'Gross USA:').next_element.next_element.strip()
    except:
        pass
    try:
        row['cum_ww_gross'] = details.find('h4', string = 'Cumulative Worldwide Gross:').next_element.next_element.strip()
    except: 
        pass
    row['production'] = [x.text.strip() for x in details.find('h4', string = 'Production Co:').find_next_siblings('a')]
#     row['runtime'] = details.time.text
#     row['sound_mix'] = [x.text.strip() for x in details.findAll(class_ = 'txt-block')[13].findAll('a')]
#     row['color'] = [x.text.strip() for x in details.findAll(class_ = 'txt-block')[14].findAll('a')]
#     row['aspect_ratio'] = details.findAll(class_ = 'txt-block')[15].contents[2].strip()
    return pd.Series(row)
    

In [417]:
scraped_df = df.apply(get_info, axis = 1)

scraping https://www.imdb.com/title/tt0111161/
scraping https://www.imdb.com/title/tt0111161/
scraping https://www.imdb.com/title/tt0068646/
scraping https://www.imdb.com/title/tt0071562/
scraping https://www.imdb.com/title/tt0468569/
scraping https://www.imdb.com/title/tt0050083/
scraping https://www.imdb.com/title/tt0108052/
scraping https://www.imdb.com/title/tt0167260/
scraping https://www.imdb.com/title/tt0110912/
scraping https://www.imdb.com/title/tt0060196/
scraping https://www.imdb.com/title/tt0137523/
scraping https://www.imdb.com/title/tt0120737/
scraping https://www.imdb.com/title/tt0109830/
scraping https://www.imdb.com/title/tt1375666/
scraping https://www.imdb.com/title/tt0080684/
scraping https://www.imdb.com/title/tt0167261/
scraping https://www.imdb.com/title/tt0073486/
scraping https://www.imdb.com/title/tt0099685/
scraping https://www.imdb.com/title/tt0133093/
scraping https://www.imdb.com/title/tt4154796/
scraping https://www.imdb.com/title/tt0047478/
scraping http

scraping https://www.imdb.com/title/tt0015324/
scraping https://www.imdb.com/title/tt0061512/
scraping https://www.imdb.com/title/tt0978762/
scraping https://www.imdb.com/title/tt0046438/
scraping https://www.imdb.com/title/tt2119532/
scraping https://www.imdb.com/title/tt0032976/
scraping https://www.imdb.com/title/tt0107290/
scraping https://www.imdb.com/title/tt0892769/
scraping https://www.imdb.com/title/tt2267998/
scraping https://www.imdb.com/title/tt3011894/
scraping https://www.imdb.com/title/tt0120382/
scraping https://www.imdb.com/title/tt0079944/
scraping https://www.imdb.com/title/tt2278388/
scraping https://www.imdb.com/title/tt0107207/
scraping https://www.imdb.com/title/tt0018455/
scraping https://www.imdb.com/title/tt0092005/
scraping https://www.imdb.com/title/tt0025316/
scraping https://www.imdb.com/title/tt0353969/
scraping https://www.imdb.com/title/tt0091763/
scraping https://www.imdb.com/title/tt0758758/
scraping https://www.imdb.com/title/tt0074958/
scraping http

In [419]:
scraped_df.head()

,budget,country,cum_ww_gross,film_location,genre,lang,plot,plot_keywords,production,rating,rating_count,release_date,runtime,stars,title,us_gross,writers
0,"$25,000,000",[USA],"$58,500,000","St. Croix, U.S. Virgin Islands",[Drama],[English],Two imprisoned men bond over a number of years...,"[wrongful imprisonment, escape from prison, ba...",[Castle Rock Entertainment],R,"2,108,229",14 October 1994 (USA),2h 22min,"[Tim Robbins, Morgan Freeman, Bob Gunton]",The Shawshank Redemption,"$28,341,469","[Stephen King, Frank Darabont]"
1,"$6,000,000",[USA],"$245,066,411","NY Eye and Ear Infirmary, 2nd Avenue & East 13...","[Crime, Drama]","[English, Italian, Latin]",The aging patriarch of an organized crime dyna...,"[mafia, crime family, patriarch, organized cri...","[Paramount Pictures, Alfran Productions]",R,"1,447,299",24 March 1972 (USA),2h 55min,"[Marlon Brando, Al Pacino, James Caan]",The Godfather,"$134,966,411,","[Mario Puzo, Francis Ford Coppola, 1 more credit]"
2,"$13,000,000",[USA],NaN,"2045 N. Hibiscus Drive, North Miami, Florida, USA","[Crime, Drama]","[English, Italian, Spanish, Latin, Sicilian]",The early life and career of Vito Corleone in ...,"[revenge, corrupt politician, bloody body of c...","[Paramount Pictures, The Coppola Company]",R,"1,006,257",20 December 1974 (USA),3h 22min,"[Al Pacino, Robert De Niro, Robert Duvall]",The Godfather: Part II,"$57,300,000","[Francis Ford Coppola, Mario Puzo, 1 more credit]"
3,"$185,000,000","[USA, UK]","$1,004,558,444,","Times Square, Causeway Bay, Hong Kong","[Action, Crime, Drama]","[English, Mandarin]",When the menace known as The Joker emerges fro...,"[dc comics, moral dilemma, psychopath, clown, ...","[Warner Bros., Legendary Entertainment, Syncopy]",PG-13,"2,074,266",18 July 2008 (USA),2h 32min,"[Christian Bale, Heath Ledger, Aaron Eckhart]",The Dark Knight,"$534,858,444,","[Jonathan Nolan, Christopher Nolan, 3 more cre..."
4,"$350,000",[USA],NaN,"New York County Courthouse - 60 Centre Street,...",[Drama],[English],A jury holdout attempts to prevent a miscarria...,"[jury, dialogue driven, courtroom, single set ...",[Orion-Nova Productions],Not Rated,"598,977",10 April 1957 (USA),1h 36min,"[Henry Fonda, Lee J. Cobb, Martin Balsam]",12 Angry Men,"$4,360,000","[Reginald Rose, Reginald Rose]"


In [426]:
len(scraped_df.columns)
scraped_df.columns

Index(['budget', 'country', 'cum_ww_gross', 'film_location', 'genre', 'lang',
       'plot', 'plot_keywords', 'production', 'rating', 'rating_count',
       'release_date', 'runtime', 'stars', 'title', 'us_gross', 'writers'],
      dtype='object')

In [439]:
scraped_df = scraped_df[['title', 'writers', 'stars', 'country', 'lang', 'film_location', 'genre', 'rating', 'rating_count', 'plot', 'plot_keywords', 'runtime', 'release_date', 'production', 'budget', 'us_gross', 'cum_ww_gross']]

scraped_df.head()


,title,writers,stars,country,lang,film_location,genre,rating,rating_count,plot,plot_keywords,runtime,release_date,production,budget,us_gross,cum_ww_gross
0,The Shawshank Redemption,"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",[USA],[English],"St. Croix, U.S. Virgin Islands",[Drama],R,"2,108,229",Two imprisoned men bond over a number of years...,"[wrongful imprisonment, escape from prison, ba...",2h 22min,14 October 1994 (USA),[Castle Rock Entertainment],"$25,000,000","$28,341,469","$58,500,000"
1,The Godfather,"[Mario Puzo, Francis Ford Coppola, 1 more credit]","[Marlon Brando, Al Pacino, James Caan]",[USA],"[English, Italian, Latin]","NY Eye and Ear Infirmary, 2nd Avenue & East 13...","[Crime, Drama]",R,"1,447,299",The aging patriarch of an organized crime dyna...,"[mafia, crime family, patriarch, organized cri...",2h 55min,24 March 1972 (USA),"[Paramount Pictures, Alfran Productions]","$6,000,000","$134,966,411,","$245,066,411"
2,The Godfather: Part II,"[Francis Ford Coppola, Mario Puzo, 1 more credit]","[Al Pacino, Robert De Niro, Robert Duvall]",[USA],"[English, Italian, Spanish, Latin, Sicilian]","2045 N. Hibiscus Drive, North Miami, Florida, USA","[Crime, Drama]",R,"1,006,257",The early life and career of Vito Corleone in ...,"[revenge, corrupt politician, bloody body of c...",3h 22min,20 December 1974 (USA),"[Paramount Pictures, The Coppola Company]","$13,000,000","$57,300,000",NaN
3,The Dark Knight,"[Jonathan Nolan, Christopher Nolan, 3 more cre...","[Christian Bale, Heath Ledger, Aaron Eckhart]","[USA, UK]","[English, Mandarin]","Times Square, Causeway Bay, Hong Kong","[Action, Crime, Drama]",PG-13,"2,074,266",When the menace known as The Joker emerges fro...,"[dc comics, moral dilemma, psychopath, clown, ...",2h 32min,18 July 2008 (USA),"[Warner Bros., Legendary Entertainment, Syncopy]","$185,000,000","$534,858,444,","$1,004,558,444,"
4,12 Angry Men,"[Reginald Rose, Reginald Rose]","[Henry Fonda, Lee J. Cobb, Martin Balsam]",[USA],[English],"New York County Courthouse - 60 Centre Street,...",[Drama],Not Rated,"598,977",A jury holdout attempts to prevent a miscarria...,"[jury, dialogue driven, courtroom, single set ...",1h 36min,10 April 1957 (USA),[Orion-Nova Productions],"$350,000","$4,360,000",NaN


In [434]:
merged_df = df.merge(scraped_df, left_index = True, right_index = True)

In [435]:
merged_df.to_csv('IMDb_Top250_moreinfo.csv', index = False)

In [437]:
scraped_df.to_csv('IMDb_Top250_pages.csv', index = False)